In [53]:
import os
import glob
import platform
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.utils.data as data_utils
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as opti
from torch.autograd import Variable

import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

import cv2
from PIL import Image

import numpy as np
from numpy.linalg import inv as denseinv
from scipy import sparse
from scipy.sparse import lil_matrix, csr_matrix
from scipy.sparse.linalg import spsolve
from scipy.sparse.linalg import inv as spinv

# Configurations

In [83]:
class Args(object):
    pass
args = Args()
args.arch = "densenet121"
args.image_h = 256
args.image_w = 256
args.img_extentions = ["png"]
args.debug = True
if platform.system() == "Darwin": args.train_dir = '/Volumes/Transcend/dataset/sintel2'


# My DataLoader

In [9]:
def default_loader(path):
    return Image.open(path).convert('RGB')

def make_dataset(dir):
    images_paths = glob.glob(os.path.join(dir, 'clean', '*', '*.png'))
    albedo_paths = images_paths[:]
    shading_paths = images_paths[:]
    pathes = []
    for img_path in images_paths:
        sp = img_path.split('/'); sp[-3] = 'albedo'; sp = ['/'] + sp; albedo_path = os.path.join(*sp)
        sp = img_path.split('/'); sp[-3] = 'albedo'; sp = ['/'] + sp; shading_path = os.path.join(*sp)
        pathes.append((img_path, albedo_path, shading_path))
    return pathes

class MyImageFolder(data_utils.Dataset):
    def __init__(self, root, transform=None, target_transform=None,
                loader=default_loader):
        imgs = make_dataset(root)
        if len(imgs) == 0:
            raise(RuntimeError("Found 0 images in subfolders of: " + root + "\n"
                               "Supported image extensions are: " + ",".join(args.img_extentions)))

        self.root = root
        self.imgs = imgs
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader
        
    def __getitem__(self, index):
        img_path, albedo_path, shading_path = self.imgs[index]
        
        img = self.loader(img_path)
        albedo = self.loader(albedo_path)
        shading = self.loader(shading_path)
        
        if self.transform is not None: img = self.transform(img)
        if self.transform is not None: albedo = self.transform(albedo)
        if self.transform is not None: shading = self.transform(shading)

        return img, albedo, shading
    
    def __len__(self):
        return len(self.imgs)
    
dataset= MyImageFolder(args.train_dir, 
                       transforms.Compose(
        [transforms.RandomCrop((args.image_h, args.image_w)),
         transforms.ToTensor()]
    ))

train_loader =data_utils.DataLoader(dataset,1,True,num_workers=1)

# Load Pretrained Model

[Defination](https://github.com/pytorch/vision/blob/master/torchvision/models/densenet.py)
* DenseNet-121: num_init_features=64, growth_rate=32, block_config=(6, 12, 24, 16)
    * First Convolution: 32M -> 16M -> 8M
    * every transition: 8M -> 4M -> 2M (downsample 1/2, except the last block)

In [98]:
densenet = models.__dict__[args.arch](pretrained=False)



class PreTrainedModel(nn.Module):
    def __init__(self, pretrained):
        super(PreTrainedModel, self).__init__()
        common_features_net = nn.Sequential(*list(pretrained.children())[0:1])
        self.net_16M = nn.Sequential(OrderedDict([
            ('conv0', common_features_net[0].conv0),
            ('norm0', common_features_net[0].norm0),
            ('relu0', common_features_net[0].relu0)
        ]))
        self.net_8M = nn.Sequential(OrderedDict([
            ('pool0', common_features_net[0].pool0)
        ]))
        self.net_4M = nn.Sequential(OrderedDict([
            ('denseblock1', common_features_net[0].denseblock1),
            ('transition1', common_features_net[0].transition1)
        ]))
        self.net_2M = nn.Sequential(OrderedDict([
            ('denseblock2', common_features_net[0].denseblock1),
            ('transition2', common_features_net[0].transition1)
        ]))
        self.net_1M = nn.Sequential(OrderedDict([
            ('denseblock3', common_features_net[0].denseblock1),
            ('transition3', common_features_net[0].transition1),
            ('denseblock4', common_features_net[0].transition1)
        ]))
    def forward(self, ft_32M):
        
        pretrained_features = [0]*5
        pretrained_features[0] = self.net_16M(ft_32M)
        pretrained_features[1]  = self.net_8M(ft_16M)
        pretrained_features[2]  = self.net_4M(ft_8M)
        pretrained_features[3]  = self.net_2M(ft_4M)
        pretrained_features[4]  = self.net_1M(ft_2M)
        
        return pretrained_features

if args.debug == True:
    
    print densenet
#     common_features_net = nn.Sequential(*list(densenet.children())[0:1])
#     net_x = nn.Sequential(OrderedDict([
#             ('conv0', common_features_net[0].conv0),
#     ]))
    
#     """
#         debug: copy/clone
#     """
#     print net_x
    t = nn.Sequential(*list(net_x.children()))

#     for param in _8M.parameters():
#         print param.data

#     for param in t.parameters():
#         param.data = (param.data*2)
#     print "@@@@@@"

#     for param in t.parameters():
#         print param.data


DenseNet (
  (features): Sequential (
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (relu0): ReLU (inplace)
    (pool0): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
    (denseblock1): _DenseBlock (
      (denselayer1): _DenseLayer (
        (norm.1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (relu.1): ReLU (inplace)
        (conv.1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm.2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
        (relu.2): ReLU (inplace)
        (conv.2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer (
        (norm.1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True)
        (relu.1): ReLU (inplace)
        (conv.1): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

In [95]:

class _DenseLayer(nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super(_DenseLayer, self).__init__()
        self.add_module('norm.1', nn.BatchNorm2d(num_input_features)),
        self.add_module('relu.1', nn.ReLU(inplace=True)),
        self.add_module('conv.1', nn.Conv2d(num_input_features, bn_size *
                        growth_rate, kernel_size=1, stride=1, bias=False)),
        self.add_module('norm.2', nn.BatchNorm2d(bn_size * growth_rate)),
        self.add_module('relu.2', nn.ReLU(inplace=True)),
        self.add_module('conv.2', nn.Conv2d(bn_size * growth_rate, growth_rate,
                        kernel_size=3, stride=1, padding=1, bias=False)),
        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = super(_DenseLayer, self).forward(x)
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate, training=self.training)
        return torch.cat([x, new_features], 1)


class _DenseBlock(nn.Sequential):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(num_input_features + i * growth_rate, growth_rate, bn_size, drop_rate)
            self.add_module('denselayer%d' % (i + 1), layer)

class _MyTransition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features, downsample=1):
        super(_Transition, self).__init__()
        self.add_module('norm', nn.BatchNorm2d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv2d(num_input_features, num_output_features,
                                          kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool2d(kernel_size=2, stride=downsample))
        
class GradientNet(nn.Module):
    def build_blocks(num_block, num_init_features):
        num_features = num_init_features
        features = nn.Sequential()
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(num_layers=num_layers, num_input_features=num_features,
                                bn_size=bn_size, growth_rate=growth_rate, drop_rate=drop_rate)
            features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            
            trans = _MyTransition(num_input_features=num_features, num_output_features=num_features // 2)
            features.add_module('transition%d' % (i + 1), trans)
            num_features = num_features // 2
        return features
    
    def __init__(self, pretrained_model):
        super(GradientNet, self).__init__()
        self.block_config = [(3,3,3),(6,6,6),(12,12,12),(16,16,16),(24,24,24)]
        self.num_input_features = [64,64,0,0,0]
        self.upsample_config = [2,4,8,16,32]
        self.pretrained_model = pretrained_model
        
        self.denseblocks = [0] * len(self.block_config)
        for i in range(0, len(self.block_config)):
            self.denseblocks[i] = self.build_blocks(self.block_config[i], self.num_input_features[i])
        
        self.upsamples = [0] * len(self.block_config)
        for i in range(0, len(self.block_config)):
            self.upsamples[i] = nn.ConvTranspose2d(in_channels=, out_channels=, kernel_size=, 
                                    stride=, padding=, output_padding=, groups=, bias=, dilation=)
        
        self.merge = nn.Sequential()
        for i in range(0,3): 
            self.merge.add_module(nn.Conv2d(in_channels=, out_channels=, kernel_size=1,
                                      stride=, padding=, groups=, bias=, dilation=))
                                    
    def forward(self, ft_input):
        ft_pretrained = self.pretrained_model(ft_input)
        ft_predict = [b(ft_pretrained[i]) for i, b in self.denseblocks]
        ft_upsampled = [up(ft_predict[i]) for i, up in self.upsamples]
        ft_concated = torch.cat(ft_upsampled, 1)
        ft_merged = merge(ft_concated)
        return ft_predict, ft_merged


SyntaxError: invalid syntax (<ipython-input-95-8ae601b6560a>, line 35)

In [93]:
x = models.DenseNet._DenseBlock(1,1,1,0,0)

AttributeError: type object 'DenseNet' has no attribute '_DenseBlock'

In [ ]:
# My nn Modules
class PretainedModel(nn.Module):
    def __init__(self, original_model, arch):
        if arch == "densenet121":
            

In [ ]:
# Visualize Graph

In [4]:
from graphviz import Digraph
import torch
from torch.autograd import Variable


def make_dot(var, params=None):
    """ Produces Graphviz representation of PyTorch autograd graph

    Blue nodes are the Variables that require grad, orange are Tensors
    saved for backward in torch.autograd.Function

    Args:
        var: output Variable
        params: dict of (name, Variable) to add names to node that
            require grad (TODO: make optional)
    """
    if params is not None:
        assert isinstance(params.values()[0], Variable)
        param_map = {id(v): k for k, v in params.items()}

    node_attr = dict(style='filled',
                     shape='box',
                     align='left',
                     fontsize='12',
                     ranksep='0.1',
                     height='0.2')
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()

    def size_to_str(size):
        return '('+(', ').join(['%d' % v for v in size])+')'

    def add_nodes(var):
        if var not in seen:
            if torch.is_tensor(var):
                dot.node(str(id(var)), size_to_str(var.size()), fillcolor='orange')
            elif hasattr(var, 'variable'):
                u = var.variable
                name = param_map[id(u)] if params is not None else ''
                node_name = '%s\n %s' % (name, size_to_str(u.size()))
                dot.node(str(id(var)), node_name, fillcolor='lightblue')
            else:
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, 'next_functions'):
                for u in var.next_functions:
                    if u[0] is not None:
                        dot.edge(str(id(u[0])), str(id(var)))
                        add_nodes(u[0])
            if hasattr(var, 'saved_tensors'):
                for t in var.saved_tensors:
                    dot.edge(str(id(t)), str(id(var)))
                    add_nodes(t)
    add_nodes(var.creator)
    return dot

ImportError: No module named graphviz

In [ ]:
make